Bạn có thể viết một bộ kiểm thử cho lớp `ThyroidCancerDataLoader` để đảm bảo rằng các hàm khởi tạo và phương thức `get_data_loader` hoạt động đúng. Dưới đây là một ví dụ về cách thực hiện điều này bằng cách sử dụng `unittest`.

### Giải thích

1. **`setUp` và `tearDown`:**
   - Tạo một thư mục tạm thời và các thư mục con để lưu các ảnh giả phục vụ cho việc kiểm thử.
   - Định nghĩa các phép biến đổi.
   - Khởi tạo một instance của `ThyroidCancerDataLoader`.
   - Sau khi kiểm thử xong, thư mục tạm thời sẽ được dọn dẹp.

2. **`test_get_data_loader`:**
   - Kiểm thử `DataLoader` cho tập huấn luyện, xác thực, và kiểm thử.
   - Đảm bảo rằng các batch từ `DataLoader` có kích thước đúng và dữ liệu trả về có kiểu dữ liệu đúng.

### Chạy bộ kiểm thử

Để chạy bộ kiểm thử này, bạn chỉ cần lưu đoạn mã trên vào một file Python (ví dụ `test_thyroid_cancer_data_loader.py`) và chạy file này:

```bash
python test_thyroid_cancer_data_loader.py
```

Lưu ý rằng bạn cần chắc chắn rằng các thư viện cần thiết (như `unittest`, `torch`, `torchvision`, `PIL`) đã được cài đặt.

In [ ]:
%pip install unittest tempfile

In [ ]:
import unittest
import tempfile
import os
from PIL import Image
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from src.data_preparation.ThyroidCancerDataset import ThyroidCancerDataset
from src.data_preparation.ThyroidCancerDataLoader import ThyroidCancerDataLoader

class TestThyroidCancerDataLoader(unittest.TestCase):

    def setUp(self):
        # Tạo thư mục tạm thời và các thư mục con chứa ảnh giả
        self.test_dir = tempfile.TemporaryDirectory()
        os.makedirs(os.path.join(self.test_dir.name, 'B2'))
        os.makedirs(os.path.join(self.test_dir.name, 'B5'))
        os.makedirs(os.path.join(self.test_dir.name, 'B6'))

        for subdir in ['B2', 'B5', 'B6']:
            for i in range(3):
                img = Image.new('RGB', (100, 100))
                img.save(os.path.join(self.test_dir.name, subdir, f'{subdir}_{i}.jpg'))

        # Định nghĩa phép biến đổi
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.RandomCrop(224),
            transforms.RandomRotation(180),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor()
        ])

        # Tạo instance của ThyroidCancerDataLoader
        self.data_loader = ThyroidCancerDataLoader(transform=self.transform, balance=True)

    def tearDown(self):
        # Dọn dẹp thư mục tạm thời sau khi kiểm thử xong
        self.test_dir.cleanup()

    def test_get_data_loader(self):
        # Kiểm thử DataLoader cho tập huấn luyện
        train_loader = self.data_loader.get_data_loader(img_dir=self.test_dir.name, mode='train', batch_size=2, num_workers=2)
        for images, labels in train_loader:
            self.assertEqual(images.shape[0], 2)
            self.assertIsInstance(images, torch.Tensor)
            self.assertIsInstance(labels, torch.Tensor)
            break  # Chỉ kiểm tra batch đầu tiên

        # Kiểm thử DataLoader cho tập xác thực
        val_loader = self.data_loader.get_data_loader(img_dir=self.test_dir.name, mode='val', batch_size=2, num_workers=2)
        for images, labels in val_loader:
            self.assertEqual(images.shape[0], 2)
            self.assertIsInstance(images, torch.Tensor)
            self.assertIsInstance(labels, torch.Tensor)
            break  # Chỉ kiểm tra batch đầu tiên

        # Kiểm thử DataLoader cho tập kiểm thử
        test_loader = self.data_loader.get_data_loader(img_dir=self.test_dir.name, mode='test', batch_size=2, num_workers=2)
        for images, labels in test_loader:
            self.assertEqual(images.shape[0], 2)
            self.assertIsInstance(images, torch.Tensor)
            self.assertIsInstance(labels, torch.Tensor)
            break  # Chỉ kiểm tra batch đầu tiên

if __name__ == '__main__':
    unittest.main()